<a href="https://colab.research.google.com/github/openUC2/UC2-REST/blob/master/DOCUMENTATION/DOC_UC2Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

# UC2 REST Tutorial

Here we are going to teach you how to interact with the UC2 microcontroller and how you can add additional functionalities. 

In order to use the client in your python environment you need the following packages to be installed:

(use the `!` to install it from within this jupyter notebook)
```py
!pip install UC2-REST
```

This code has been tested with the ESP32 WEMOS D1 R32 + CNC shield v3, where 3 stepper are connected to the board and an LED Matrix (WS2812, adafruit) is connected to the FEED pin. 

If you find an error, please report it [here](https://github.com/openUC2/UC2-REST/issues/new) 

## Use in your local Jupyter notebook 

### Step 1: Install Jupyter 

Install [Jupyter](http://jupyter.org/install) on your local machine.

### Step 2: Download/Clone this repository

E.g. `git clone https://github.com/openUC2/UC2-REST/`

### Step 3: Open this notebook in Jupyter 

open a terminal and change into this folder e.g.:

```
cd YOURDOWNLOADPATH/UC2-REST/DOCUMENTATION/
jupyter notebook 
```

Open the notebook from the jupyter fileexplorer. 

## Use this in Google COLAB 

You can use the UC2-Board without extra-complicated installation routines. For this, you need to link your Jupyter Notebook to the Colab plugin. From the official documentation ([here](https://research.google.com/colaboratory/local-runtimes.html)), please follow these steps: 

In order to allow Colaboratory to connect to your locally running Jupyter server, you'll need to perform the following steps.

### Step 1: Install Jupyter
Install [Jupyter](http://jupyter.org/install) on your local machine.

### Step 2: Install and enable the jupyter_http_over_ws jupyter extension (one-time)
The jupyter_http_over_ws extension is authored by the Colaboratory team and available on GitHub.

```
pip install jupyter_http_over_ws
jupyter serverextension enable --py jupyter_http_over_ws
```

### Step 3: Start server and authenticate

New notebook servers are started normally, though you will need to set a flag to explicitly trust WebSocket connections from the Colaboratory frontend.

```
jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0
```

Once the server has started, it will print a message with the initial backend URL used for authentication. Make a copy of this URL as you'll need to provide this in the next step.

### Step 4: Connect to the local runtime

In Colaboratory, click the "Connect" button and select "Connect to local runtime...". Enter the URL from the previous step in the dialog that appears and click the "Connect" button. After this, you should now be connected to your local runtime.



In [1]:
# Install the latest library
!pip install UC2-REST # only if you have not installed it already

## Organize all imports

First of all we need to import the `ESP32Client`. Since it is not yet a standalone pip package, we have to do that via a relaitve import, meaning that the file is in the same folder as this Jupyter Notebook

In [2]:
%load_ext autoreload 
%autoreload 2


import uc2rest
import time
import numpy as np

# Flashing the Firmware

You have three options to get the latest firmware:

1. Use the Web-based update tool which you can find here: https://youseetoo.github.io/
2. Use the Python firmware update tool that is also hosted in this repository; Drawback: It seems to not work consistently accross different Windows versions. 
3. Download/Clone this repo and compile it yourself to finally upload it to your MCU

# Connecting to the ESP32 via USB

Now we want to initiliaze the USB-serial connection. Therefore, connect the ESP32 to your computer using a data (!) USB cable and establish the connection. You can leave the port as "unknown" as the portfinder may identify the ESP.

**Important:** Close all applications that may be connected to the ESP (e.g. Arduino Serial Plotter)

**IMPORTANT:** Install the USB serial driver for the ESP32: https://learn.sparkfun.com/tutorials/how-to-install-ch340-drivers/all

In [5]:
ESP32 = uc2rest.UC2Client(serialport="unknown")

# setting debug output of the serial to true - all message will be printed
ESP32.serial.DEBUG=False

2022-12-16 11:32:48 ERROR [UC2Client] [Errno 2] could not open port unknown: [Errno 2] No such file or directory: 'unknown'
2022-12-16 11:32:48 DEBUG [UC2Client] /dev/cu.Bluetooth-Incoming-Port
2022-12-16 11:32:48 DEBUG [UC2Client] /dev/cu.usbserial-0001
2022-12-16 11:32:53 DEBUG [UC2Client] We are connected: True on port: /dev/cu.usbserial-0001
2022-12-16 11:32:53 DEBUG [UC2Client] Using API version 2
2022-12-16 11:32:53 DEBUG [UC2Client] Attention, lasers are on channels 1,2,3


loaded config is valid


# Test the REST via Serial

We can test the serial and check which firmware was uploaded. Most commands return a dictionary, hence you can retrieve the value by accessing the `dict`

In [8]:
# get motor response
test_cmd = "{'task': '/motor_get'}"
ESP32.serial.writeSerial(test_cmd)
cmd_return = ESP32.serial.readSerial()
print(cmd_return)

# get state response
test_cmd = "{'task': '/state_get'}"
ESP32.serial.writeSerial(test_cmd)
cmd_return = ESP32.serial.readSerial()
print(cmd_return)

print("The Firmware version is: "+str(cmd_return['identifier_date']))

{'steppers': [{'stepperid': 0, 'dir': 0, 'step': 0, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': True, 'speed': 0, 'speedmax': 200000, 'max_pos': 0, 'min_pos': 0, 'position': 0}, {'stepperid': 1, 'dir': 16, 'step': 26, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': True, 'speed': 0, 'speedmax': 200000, 'max_pos': 0, 'min_pos': 0, 'position': 0}, {'stepperid': 2, 'dir': 27, 'step': 25, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': True, 'speed': 0, 'speedmax': 200000, 'max_pos': 0, 'min_pos': 0, 'position': 0}, {'stepperid': 3, 'dir': 14, 'step': 17, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': True, 'speed': 0, 'speedmax': 200000, 'max_pos': 0, 'min_pos': 0, 'position': 0}]}
{'identifier_name': 'UC2_Feather', 'identifier_id': 'V2.0', 'identifier_date': 'Dec 15 202223:22:22', 'identifier_author': 'BD', 'IDENTIFIER_NAME': 'uc2-esp', 'configIsSet': False

# Modules 

Different modules can be activated/deactivated. 

In [10]:
''' ################
MODULES
################'''
mModules = ESP32.modules.get_default_modules()
print(mModules)
mModulesDevice = ESP32.modules.get_modules()
print(mModulesDevice)
ESP32.modules.set_modules(mModules)
# wait for reboot
time.sleep(2)
mModulesDevice = ESP32.modules.get_modules()
print("The activated modules are: "+mModulesDevice)


{'led': 1, 'motor': 1, 'slm': 0, 'home': 0, 'analogin': 0, 'pid': 0, 'laser': 1, 'dac': 0, 'analogout': 0, 'digitalout': 0, 'digitalin': 1, 'scanner': 0, 'wifi': 0}
  
The activated modules are:   


# Moving the motor 

The following code snippets will help you moving the motors (XYZ) continously or at a known number of `steps` at a certain `speed` level (all measured in steps/s). 

The additional attributs 
- `is_blocking` states if the action is performed in the background or not; if `False` no return message will be provided
- `is_absolute` says if we go relative or absolute steps 
- `is_enabled` says if we want to "unpower" the motors once we are done (prevent overheating)


Once the microcontroller (ESP32) is flashed, the pin definitions are not set yet. We have to do that manually using a customized dictionary and write that to the MCU.

**In general:**
 The axes are:
 A => 0
 X => 1
 Y => 2
 Z => 3

In [11]:
if 0: # we don't want to change the configuration now! 
    # setup all motors at once 
    print(ESP32.motor.settingsdict)

    # Remember => 0 = A, 1 = X, 2 = Y, 3 = Z w.r.t. the axises of the microscope

    ESP32.motor.settingsdict["motor"]["steppers"][0]["dir"]=18
    ESP32.motor.settingsdict["motor"]["steppers"][0]["step"]=19
    ESP32.motor.settingsdict["motor"]["steppers"][0]["enable"]=12

    ESP32.motor.settingsdict["motor"]["steppers"][1]["dir"]=16
    ESP32.motor.settingsdict["motor"]["steppers"][1]["step"]=26
    ESP32.motor.settingsdict["motor"]["steppers"][2]["dir"]=27

    ESP32.motor.settingsdict["motor"]["steppers"][2]["step"]=25
    ESP32.motor.settingsdict["motor"]["steppers"][3]["dir"]=14
    ESP32.motor.settingsdict["motor"]["steppers"][3]["step"]=17

    ESP32.motor.settingsdict["motor"]["steppers"][1]["enable"]=12
    ESP32.motor.settingsdict["motor"]["steppers"][2]["enable"]=12
    ESP32.motor.settingsdict["motor"]["steppers"][3]["enable"]=12
    ESP32.motor.set_motors(ESP32.motor.settingsdict)

# check if we set the right parameters
ESP32.motor.get_motors()


[{'stepperid': 0,
  'dir': 0,
  'step': 0,
  'enable': 12,
  'dir_inverted': False,
  'step_inverted': False,
  'enable_inverted': True,
  'speed': 0,
  'speedmax': 200000,
  'max_pos': 0,
  'min_pos': 0,
  'position': 0},
 {'stepperid': 1,
  'dir': 16,
  'step': 26,
  'enable': 12,
  'dir_inverted': False,
  'step_inverted': False,
  'enable_inverted': True,
  'speed': 0,
  'speedmax': 200000,
  'max_pos': 0,
  'min_pos': 0,
  'position': 0},
 {'stepperid': 2,
  'dir': 27,
  'step': 25,
  'enable': 12,
  'dir_inverted': False,
  'step_inverted': False,
  'enable_inverted': True,
  'speed': 0,
  'speedmax': 200000,
  'max_pos': 0,
  'min_pos': 0,
  'position': 0},
 {'stepperid': 3,
  'dir': 14,
  'step': 17,
  'enable': 12,
  'dir_inverted': False,
  'step_inverted': False,
  'enable_inverted': True,
  'speed': 0,
  'speedmax': 200000,
  'max_pos': 0,
  'min_pos': 0,
  'position': 0}]

In [12]:
if 0: # we don't want to change the configuration now
    # OR setup motors individually (according to WEMOS R32 D1)
    ESP32.motor.set_motor(stepperid = 1, position = 0, stepPin = 26, dirPin=16, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 2, position = 0, stepPin = 25, dirPin=27, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 3, position = 0, stepPin = 17, dirPin=14, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)
    ESP32.motor.set_motor(stepperid = 0, position = 0, stepPin = 19, dirPin=18, enablePin=12, maxPos=None, minPos=None, acceleration=None, isEnable=1)

# get individual motors
print(ESP32.motor.get_motor(axis = 1))


{'stepperid': 1, 'dir': 16, 'step': 26, 'enable': 12, 'dir_inverted': False, 'step_inverted': False, 'enable_inverted': True, 'speed': 0, 'speedmax': 200000, 'max_pos': 0, 'min_pos': 0, 'position': 0}


We can also setup individual motor settings like so:

In [13]:
ESP32.motor.set_motor_currentPosition(axis=0, currentPosition=10000)
ESP32.motor.set_motor_acceleration(axis=0, acceleration=10000)
ESP32.motor.set_motor_enable(is_enable=1)
ESP32.motor.set_direction(axis=1, sign=1, timeout=1)
ESP32.motor.set_position(axis=1, position=0, timeout=1)

# wait to settle
time.sleep(2)

## Moving the motors

With the below code, we can test the motors spinning or not

In [15]:
position1 = ESP32.motor.get_position(timeout=1)
print(position1)
ESP32.motor.move_x(steps=10000, speed=10000, is_blocking=True)
ESP32.motor.move_y(steps=1000, speed=1000, is_blocking=True)
ESP32.motor.move_z(steps=1000, speed=1000, is_blocking=True)
ESP32.motor.move_t(steps=1000, speed=1000)
ESP32.motor.move_xyzt(steps=(0,10000,10000,0), speed=10000, is_blocking=True)
ESP32.motor.move_xyzt(steps=(0,0,0,0), speed=10000, is_absolute=True, is_blocking=True)
ESP32.motor.move_forever(speed=(0,100,0,0), is_stop=False)
time.sleep(1)
ESP32.motor.move_forever(speed=(0,0,0,0), is_stop=True)

position2 = ESP32.motor.get_position(timeout=1)
print(position2)

[0 0 0 0]
[    0 18418  9418    -1]


# Drive the motor in a scanning grid

In [ ]:
dDist = 1000
speed = 20000
nDist = 4

# test Motor in scanning mode
ESP32.motor.move_xyzt(steps=(0,0,0,0), speed=speed, is_absolute = True, is_blocking=True)

for ix in range(nDist):
    for iy in range(nDist):
        if ix%2==0:
            iy=nDist-iy
        ESP32.motor.move_xyzt(steps=(0,ix*dDist,iy*dDist,0), speed=speed, is_absolute = True, is_blocking=True)
ESP32.motor.move_xyzt(steps=(0,nDist*dDist,nDist*dDist,0), speed=speed, is_absolute = True, is_blocking=True)
ESP32.motor.move_xyzt(steps=(0,0,0,0), speed=speed, is_absolute = True, is_blocking=True)


# ESP32 State

In [17]:
# test state
_state = ESP32.state.get_state()
print(_state)
ESP32.state.set_state(debug=False)
_mode = ESP32.state.isControllerMode()
print(_mode)
ESP32.state.espRestart() # restarts the microcontroller
time.sleep(5)
ESP32.state.setControllerMode(isController=True)
_busy = ESP32.state.isBusy()
print(_busy)
_state = ESP32.state.get_state()
print(_state)
_state = ESP32.state.get_state()
print(_state)


{'identifier_name': 'UC2_Feather', 'identifier_id': 'V2.0', 'identifier_date': 'Dec 15 202223:22:22', 'identifier_author': 'BD', 'IDENTIFIER_NAME': 'uc2-esp', 'configIsSet': False}


# LED Matrix

If the LED matrix is connected to pin 4 and has 16 individual WS2810 LEds, you can set it up like so

In [21]:
# test LED
print("The LED pin is: "+str(ESP32.led.get_ledpin()))

ESP32.led.send_LEDMatrix_full(intensity=(255, 255, 255))
time.sleep(0.5)
ESP32.led.send_LEDMatrix_full(intensity=(0, 0, 0))

2022-12-16 11:46:43 DEBUG [UC2Client] Setting LED Pattern (full): (255, 255, 255)


The LED pin is: {'ledArrNum': 64, 'ledArrPin': 4}


2022-12-16 11:46:43 DEBUG [UC2Client] Setting LED Pattern (full): (0, 0, 0)


{'led_array': {'isDone': 0}}

In [22]:
# display random pattern
for i in range(10):
    led_pattern = np.random.randint(0,55, (25,3))
    ESP32.led.send_LEDMatrix_array(led_pattern=led_pattern,timeout=1)

ESP32.led.send_LEDMatrix_single(indexled=0, intensity=(0, 255, 0))

2022-12-16 11:49:29 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:29 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:30 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:30 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:30 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:30 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:31 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:31 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:31 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:32 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:49:32 DEBUG [UC2Client] Setting LED Pattern (single) 


{'led_array': {'isDone': 0}}

In [23]:
#%% left half / right half
led_pattern = np.zeros((25,3))
list_left = (0,1,2,3,4,5,9,10,11,12,13,14,15,16,17)
list_right = (0,5,6,7,8,9,18,19,20,21,22,23,24)
led_pattern[list_left,0] = 255
led_pattern[list_right,1] = 255
ESP32.led.send_LEDMatrix_array(led_pattern=led_pattern, timeout=1)
time.sleep(1)
ESP32.led.send_LEDMatrix_array(led_pattern=led_pattern*0, timeout=1)


2022-12-16 11:50:01 DEBUG [UC2Client] Setting LED Pattern (array) 
2022-12-16 11:50:03 DEBUG [UC2Client] Setting LED Pattern (array) 


{'led_array': {'isDone': 0}}

# Wifi
This feature will be implemented soon. It will help you to connect to a common Wifi hotspot or create an access point 

In [24]:
ESP32.wifi.scanWifi()

{'ssid': ['BueroJena',
  'BVB-GAST',
  'Andreas 7362 SL',
  'BenMur',
  'TP-LINK_FCFF',
  'Vodafone Homespot',
  'Vodafone Hotspot',
  'Vodafone-C54C',
  'Vodafone Hotspot']}

# Lasers

Lasers are essentially PWM pins and can tune a voltage signal between 0..3.3V in case of the ESP32
You can also hook up LEDs 

In [28]:
# set laser pins 
if(0):
    ESP32.laser.set_laserpin(laserid=1, laserpin=15)
    ESP32.laser.set_laserpin(laserid=2, laserpin=16)
    ESP32.laser.set_laserpin(laserid=3, laserpin=17)

# get laser pins
print(ESP32.laser.get_laserpins())
print(ESP32.laser.get_laserpin(laserid=1))

{'LASER1pin': 18, 'LASER2pin': 19, 'LASER3pin': 0}
18


In [26]:
# set laser values
ESP32.laser.set_laser(channel=1, value=1000, despeckleAmplitude=0, despecklePeriod=10, timeout=20, is_blocking = True)
ESP32.laser.set_laser(channel=2, value=1000, despeckleAmplitude=0, despecklePeriod=10, timeout=20, is_blocking = True)
ESP32.laser.set_laser(channel=3, value=1000, despeckleAmplitude=0, despecklePeriod=10, timeout=20, is_blocking = True)


2022-12-16 11:50:45 DEBUG [UC2Client] Setting Laser 1, value: 1000
2022-12-16 11:50:46 DEBUG [UC2Client] Setting Laser 2, value: 1000
2022-12-16 11:50:47 DEBUG [UC2Client] Setting Laser 3, value: 1000


'[413109][I][SerialProcess.cpp:41] loop(): process single task[413109][I][LaserController.cpp:101] act(): LaserID 3, LaserVal 1000, LaserDespeckle 0, LaserDespecklePeriod 100'